In [119]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
# from tensorflow.keras.utils import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np


In [120]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [121]:
def createdataframe(dir):
    image_paths =[]
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths,labels
        

In [122]:
train = pd.DataFrame()
train['image'],train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [123]:
train

,image,label
0,images/train\angry\0.jpg,angry
1,images/train\angry\1.jpg,angry
2,images/train\angry\10.jpg,angry
3,images/train\angry\10002.jpg,angry
4,images/train\angry\10016.jpg,angry
...,...,...
28816,images/train\surprise\9969.jpg,surprise
28817,images/train\surprise\9985.jpg,surprise
28818,images/train\surprise\9990.jpg,surprise
28819,images/train\surprise\9992.jpg,surprise


In [124]:
test = pd.DataFrame()
test['image'],test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [125]:
test

,image,label
0,images/test\angry\10052.jpg,angry
1,images/test\angry\10065.jpg,angry
2,images/test\angry\10079.jpg,angry
3,images/test\angry\10095.jpg,angry
4,images/test\angry\10121.jpg,angry
...,...,...
7061,images/test\surprise\9806.jpg,surprise
7062,images/test\surprise\9830.jpg,surprise
7063,images/test\surprise\9853.jpg,surprise
7064,images/test\surprise\9878.jpg,surprise


In [126]:
from tqdm.notebook import tqdm

In [127]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale=True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [128]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

C:\Python311\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [129]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [130]:
x_train = train_features/255.0
x_test = test_features/255.0

In [131]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [132]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [133]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [134]:
model = Sequential()

# 1st CNN Layer
model.add(Conv2D(128,kernel_size= (3, 3),activation=('relu'), padding='same', input_shape=(48,48,1)))
# model.add()
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# 2nd CNN Layer
model.add(Conv2D(256,kernel_size= (3, 3),activation=('relu'), padding='same', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# 3rd CNN Layer
model.add(Conv2D(512,kernel_size= (3, 3),activation=('relu'), padding='same', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# 4th CNN layer
model.add(Conv2D(256,kernel_size= (3, 3),activation=('relu'), padding='same', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
#fully connected layers
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))

#output Layer
model.add(Dense(7, activation = 'softmax' ))



C:\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [135]:
model.compile(optimizer = 'adam' ,loss = 'categorical_crossentropy' , metrics=['accuracy'])

In [85]:
model.fit(x=x_train,y= y_train, batch_size =128, epochs =100, validation_data =(x_test,y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 260s 1s/step - accuracy: 0.2371 - loss: 1.8366 - val_accuracy: 0.2637 - val_loss: 1.7982
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 248s 1s/step - accuracy: 0.2491 - loss: 1.7936 - val_accuracy: 0.3095 - val_loss: 1.7069
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 399s 2s/step - accuracy: 0.2973 - loss: 1.7149 - val_accuracy: 0.3814 - val_loss: 1.5959
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 404s 2s/step - accuracy: 0.3694 - loss: 1.6012 - val_accuracy: 0.4359 - val_loss: 1.4352
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 369s 2s/step - accuracy: 0.4092 - loss: 1.5051 - val_accuracy: 0.4608 - val_loss: 1.3766
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 354s 2s/step - accuracy: 0.4349 - loss: 1.4533 - val_accuracy: 0.4851 - val_loss: 1.3377
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 350s 2s/step - accuracy: 0.4575 - loss: 1.4024 - val_accuracy: 0.5067 - val_loss: 1.2768
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 357s 2s/step - accuracy: 0.4750 - loss: 1.3602 - 

KeyboardInterrupt: 

In [136]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [137]:
from keras.models import model_from_json

In [138]:
json_file = open("emotiondetector.json","r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [139]:
label = ['angry','disgust','fear','happy','neutral','sad','suprise']

In [140]:
def ef(image):
    img = load_img(image,grayscale=True)
    feature = np.array(img)
    feature.reshape(1,48,48,1)
    return feature/255.0

In [142]:

 
image = 'images/train/angry/1.jpg'
img =ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ", pred_label)

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 48), dtype=float32). Expected shape (None, 48, 48, 1), but input has incompatible shape (32, 48)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 48), dtype=float32)
  • training=False
  • mask=None